# Сверточная сеть для MNIST датасета (первый победитель на соревнованиях)

In [1]:
import numpy as np
import torch
import time
import platform

In [2]:
print(f'Pytorch: {torch.__version__}')
print(f'cuda: {torch.version.cuda}')
print(f'Python: {platform.python_version()}')

Pytorch: 2.2.1+cpu
cuda: None
Python: 3.11.6


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


### Загрузим данные и нормализуем их

In [4]:
from MNISTtools import load, show

In [6]:
xtrain, ltrain = load(dataset='training', path='dataset/')
xtest, ltest = load(dataset='testing', path='dataset/')

In [7]:
def normalize_MNIST_images(x):
    '''
    Нормализует изображения MNIST.

    Аргументы:
        x: данные

    Возвращает:
        Нормализованные данные.
    '''
    x_norm = x.astype(np.float32)
    return x_norm * 2 / 255 - 1

In [8]:
# Нормализация
xtrain = normalize_MNIST_images(xtrain)
xtest = normalize_MNIST_images(xtest)

### Изменение формы данных

Torch ожидает, что входные данные сверточного слоя будут храниться в следующем формате:
`Размер пакета × Количество входных каналов × Ширина изображения × Высота изображения`

In [9]:
xtrain = xtrain.reshape([28, 28, -1])[:, :, None, :]
xtest = xtest.reshape([28, 28, -1])[:, :, None, :]
print(f'Форма xtrain после изменения формы: {xtrain.shape}.')
print(f'Форма xtest после изменения формы: {xtest.shape}.')

Форма xtrain после изменения формы: (28, 28, 1, 60000).
Форма xtest после изменения формы: (28, 28, 1, 10000).


In [10]:
xtrain = np.moveaxis(xtrain, (2, 3), (1, 0))
xtest = np.moveaxis(xtest, (2, 3), (1, 0))
print(f'Форма xtrain после moveaxis: {xtrain.shape}.')
print(f'Форма xtest после moveaxis: {xtest.shape}.')

Форма xtrain после moveaxis: (60000, 1, 28, 28).
Форма xtest после moveaxis: (10000, 1, 28, 28).


### Обернем данные в тензор

In [11]:
xtrain = torch.from_numpy(xtrain)
ltrain = torch.from_numpy(ltrain)
xtest = torch.from_numpy(xtest)
ltest = torch.from_numpy(ltest)

In [12]:
xtrain_gpu = xtrain.to(device)
ltrain_gpu = ltrain.to(device)
xtest_gpu = xtest.to(device)
ltest_gpu = ltest.to(device)

## LeNet 5

* Сверточные слои могут быть созданы как `nn.Conv2d(N, C, K)`. Для входных изображений размером `W×H`, выходные карты признаков имеют размер `[W−K+1]x[H−K+1]`.

* Максимальное объединение реализуется так же, как и любая другая нелинейная функция (например, ReLU или softmax). Для входных изображений размером `W×H`, выходные карты признаков имеют размер `[W/L]×[H/L]`.

* Полносвязанный слой может быть создан как `nn.Linear(M, N)`.

Архитектура:

1. Сверточный слой, соединяющий входное изображение с `6` картами признаков с `5×5` свертками (`K=5`), за которым следуют ReLU и максимальное объединение (`L=2`).
2. Сверточный слой, соединяющий `6` входных каналов с `16` выходными каналами с `5×5` свертками, за которым следуют ReLU и максимальное объединение (`L=2`).
3. Полносвязанный слой, соединяющий `16` карт признаков с `120` выходными блоками, за которым следует ReLU.
4. Полносвязанный слой, соединяющий `120` входов с `84` выходными блоками, за которым следует ReLU.
5. Финальный линейный слой, соединяющий `84` входа с `10` линейными выходами (по одному для каждой из наших цифр).

Первый слой:
* Вход: `(28, 28, 1)`
* После *паддинга*: `(32, 32, 1)`
* После свертки (ядра=`5x5`): `(28, 28, 6)` где `28=32-5+1`
* После ReLU: `(28, 28, 6)`
* После максимального объединения (шаг=`2x2`): `(14, 14, 6)` $\Rightarrow$ **ВЫХОД**

Второй слой:
* Вход: `(14, 14, 6)`
* После свертки (ядра=`5x5`): `(10, 10, 16)`
* После ReLU: `(10, 10, 16)`
* После максимального объединения (шаг=`2x2`): `(5, 5, 16)` $\Rightarrow$ **ВЫХОД**

Третий слой:
* Вход: `(5, 5, 16)` $\Rightarrow$ `5x5x16=400`
* После полносвязного слоя: `(120, 1)`
* После ReLU: `(120, 1)` $\Rightarrow$ **ВЫХОД**

Четвертый слой:
* Вход: `(120, 1)`
* После полносвязного слоя: `(84, 1)`
* После ReLU: `(84, 1)` $\Rightarrow$ **ВЫХОД**

Пятый слой:
* Вход: `(84, 1)`
* После полносвязного слоя: `(10, 1)`
* После ReLU: `(10, 1)` $\Rightarrow$ **ВЫХОД**

In [13]:
import torch.nn as nn
import torch.nn.functional as F

In [14]:
class LeNet(nn.Module):

    # структура сети
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        '''
        Один проход через сеть.

        Аргументы:
            x: входные данные
        '''
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        '''
        Получить количество признаков в пакете тензоров `x`.
        '''
        size = x.size()[1:]
        return np.prod(size)

### Проверим структуру модели

In [15]:
net = LeNet()
print(net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### Проверим параметры сети

In [16]:
for name, param in net.named_parameters():
    print(name, param.size(), param.requires_grad)

conv1.weight torch.Size([6, 1, 5, 5]) True
conv1.bias torch.Size([6]) True
conv2.weight torch.Size([16, 6, 5, 5]) True
conv2.bias torch.Size([16]) True
fc1.weight torch.Size([120, 400]) True
fc1.bias torch.Size([120]) True
fc2.weight torch.Size([84, 120]) True
fc2.bias torch.Size([84]) True
fc3.weight torch.Size([10, 84]) True
fc3.bias torch.Size([10]) True


### Точность без обратного распространения ошибки

In [17]:
# избегаем отслеживания градиента во время тестирования и тем самым экономим вычислительное время
with torch.no_grad():
    yinit = net(xtest)

In [18]:
_, lpred = yinit.max(1)
print(100 * (ltest == lpred).float().mean())

tensor(8.9300)


`ltest == lpred` создает тензор со значениями `0` и `1`, где `0` означает неравенство, а `1` - равенство. Поэтому `(ltest == lpred).float().mean()` означает точность, которая составляет **11.55%**.

Стохастический градиентный спуск (SGD) с перекрестной энтропией и моментумом

Кросс-энтропия в торче это композиция из softmax и стандартной кросс-энтропии

In [22]:
def backprop_deep(xtrain, ltrain, net, T, B=100, gamma=0.001, rho=0.9):
    '''
    Обратное распространение.

    Аргументы:
        xtrain: обучающие выборки
        ltrain: тестовые выборки
        net: нейронная сеть
        T: количество эпох
        B: размер минипакета
        gamma: размер шага
        rho: импульс

    Возвращает:
        Ничего
    '''
    N = xtrain.size()[0]  # Размер набора данных для обучения
    NB = N // B  # Количество минипакетов
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=gamma, momentum=rho)

    for epoch in range(T):
        running_loss = 0.0
        shuffled_indices = np.random.permutation(NB)
        for k in range(NB):
            # Извлечение k-го минипакета из xtrain и ltrain
            minibatch_indices = range(shuffled_indices[k] * B, (shuffled_indices[k] + 1) * B)
            inputs = xtrain[minibatch_indices]
            labels = ltrain[minibatch_indices].long()


            # Инициализация градиентов нулями
            optimizer.zero_grad()

            # Прямое распространение
            outputs = net(inputs)

            # Оценка ошибки
            loss = criterion(outputs, labels)

            # Обратное распространение
            loss.backward()

            # Обновление параметров
            optimizer.step()

            # Вывод средней потери для минипакета каждые 100 минипакетов
            # Вычисление и вывод статистики
            with torch.no_grad():
                running_loss += loss.item()
            if k % 100 == 99:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, k + 1, running_loss / 100))
                running_loss = 0.0

In [23]:
net = LeNet()

In [24]:
start = time.time()
backprop_deep(xtrain, ltrain, net, T=3)
end = time.time()
print(f'{end - start:.6f} секунд на полный проход.')

[1,   100] loss: 2.302
[1,   200] loss: 2.296
[1,   300] loss: 2.289
[1,   400] loss: 2.280
[1,   500] loss: 2.262
[1,   600] loss: 2.225
[2,   100] loss: 2.123
[2,   200] loss: 1.705
[2,   300] loss: 0.911
[2,   400] loss: 0.571
[2,   500] loss: 0.442
[2,   600] loss: 0.368
[3,   100] loss: 0.339
[3,   200] loss: 0.306
[3,   300] loss: 0.270
[3,   400] loss: 0.245
[3,   500] loss: 0.240
[3,   600] loss: 0.222
17.218279 секунд на полный проход.


### Проверим тестовый датасет

In [25]:
start = time.time()
backprop_deep(xtest, ltest, net, T=3)
end = time.time()
print(f'{end - start:.6f} секунд.')

[1,   100] loss: 0.193
[2,   100] loss: 0.171
[3,   100] loss: 0.164
3.546738 секунд.


In [26]:
y = net(xtest)

In [27]:
print(100 * (ltest == y.max(1)[1]).float().mean())

tensor(95.4100)


Точность на трёх эпохах **95.41%**.

In [28]:
net_gpu = LeNet().to(device)

In [29]:
start = time.time()
backprop_deep(xtrain_gpu, ltrain_gpu, net_gpu, T=10)
end = time.time()
print(f'{end - start:.6f} секунд.')

[1,   100] loss: 2.303
[1,   200] loss: 2.300
[1,   300] loss: 2.293
[1,   400] loss: 2.286
[1,   500] loss: 2.274
[1,   600] loss: 2.253
[2,   100] loss: 2.203
[2,   200] loss: 2.019
[2,   300] loss: 1.375
[2,   400] loss: 0.788
[2,   500] loss: 0.541
[2,   600] loss: 0.432
[3,   100] loss: 0.342
[3,   200] loss: 0.306
[3,   300] loss: 0.283
[3,   400] loss: 0.257
[3,   500] loss: 0.237
[3,   600] loss: 0.190
[4,   100] loss: 0.194
[4,   200] loss: 0.183
[4,   300] loss: 0.188
[4,   400] loss: 0.174
[4,   500] loss: 0.179
[4,   600] loss: 0.163
[5,   100] loss: 0.152
[5,   200] loss: 0.163
[5,   300] loss: 0.130
[5,   400] loss: 0.140
[5,   500] loss: 0.137
[5,   600] loss: 0.134
[6,   100] loss: 0.132
[6,   200] loss: 0.131
[6,   300] loss: 0.109
[6,   400] loss: 0.118
[6,   500] loss: 0.120
[6,   600] loss: 0.126
[7,   100] loss: 0.108
[7,   200] loss: 0.120
[7,   300] loss: 0.112
[7,   400] loss: 0.101
[7,   500] loss: 0.106
[7,   600] loss: 0.108
[8,   100] loss: 0.104
[8,   200] 

### Оценим дообученную модель

In [30]:
y = net_gpu(xtest_gpu)

In [31]:
print(100 * (ltest == y.max(1)[1].cpu()).float().mean())

tensor(97.8000)


Точность с 10 эпохами **97.8%**. Напомню, что после трёх эпох мы имели точность **95.41%**.

In [32]:
start = time.time()
backprop_deep(xtest_gpu, ltest_gpu, net_gpu, T=10)
end = time.time()
print(f'{end - start:.6f} секунд.')

[1,   100] loss: 0.071
[2,   100] loss: 0.068
[3,   100] loss: 0.062
[4,   100] loss: 0.061
[5,   100] loss: 0.056
[6,   100] loss: 0.055
[7,   100] loss: 0.053
[8,   100] loss: 0.050
[9,   100] loss: 0.049
[10,   100] loss: 0.047
8.549041 секунд.
